In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request as urllib2
import tarfile
import os.path
import requests
from os import path
from bs4 import BeautifulSoup
import time

In [2]:
# Introducimos las funciones del modulo creado
from module import cpm,title

In [3]:
!pwd

/home/jovyan/work/python/python


In [4]:
rt = urllib2.urlopen("https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz")
csvzip = tarfile.open(fileobj=rt,mode='r:gz')
csvzip.extractall()

In [5]:
pos=os.listdir('aclImdb/train/pos') # List: Positive ratings of movies 
pos_url=cpm('aclImdb/train/urls_pos.txt') # List: Movie url of positive review
df_pos=pd.DataFrame({'id':pos_url,'rating':pos}) # pd.df: Dataframe of dic "rating"
df_pos.head()
regex_rating=r'\d+_(\d{1,2})\.txt' # We group the info we want to extract of the string in the first column
df_pos['rating']=df_pos['rating'].str.extract(pat=regex_rating,expand=True) # We extract the id and rating of the original review id column
regex_id=r'\S+(\d{7})\/\S+'  #'(tt\d+)' # We group the info we want to extract of the string in the first column
df_pos['id']=df_pos['id'].str.extract(pat=regex_id,expand=True) # We extract the id and rating of the original review id column
df_pos.head()

,id,rating
0,0453418,10
1,0453418,8
2,0453418,10
3,0064354,10
4,0064354,10


In [6]:
neg=os.listdir('aclImdb/train/neg') # List: Negative ratings of movies 
neg_url=cpm('aclImdb/train/urls_neg.txt') # List: Movie url of negative review
df_neg=pd.DataFrame({'id':neg_url,'rating':neg}) # pd.df: Dataframe
df_neg['rating']=df_neg['rating'].str.extract(pat=regex_rating,expand=True) # We extract the id and rating of the original review id column
df_neg['id']=df_neg['id'].str.extract(pat=regex_id,expand=True) # We extract the id (7 digits of the url)
df_neg.head()

,id,rating
0,0064354,1
1,0100680,2
2,0100680,1
3,0100680,1
4,0047200,3


In [7]:
df_pn=pd.concat([df_pos,df_neg],ignore_index=True)
print(f'rows={df_pn.shape[0]}\ncolumns={df_pn.shape[1]}') # We check the size
df_pn.head() # We get a preview of the index:id of the movie

rows=25000
columns=2


,id,rating
0,0453418,10
1,0453418,8
2,0453418,10
3,0064354,10
4,0064354,10


In [8]:
df_pn['rating']=df_pn['rating'].astype(float)
df_pn.dtypes

id         object
rating    float64
dtype: object

In [9]:
df_top = df_pn.groupby('id')['rating'].mean().reset_index()
df_top = df_top.sort_values(by='rating', ascending=False)#.reset_index(drop=True)
#print(f'Tamaño del df con el promedio de ratings por pelicula:{df_top.shape}')
best_10=df_top.head(10)
worst_10=df_top.tail(10)
#df_top['rating'].unique()

In [10]:
url_base = "http://www.imdb.com/title/tt"
id_best=best_10['id'] # id of the titles of the top 10 movies
#delay = 3  # 3-second delay between requests
for i in id_best:
    url=url_base+i # imdb url of the movies
    tt=title(url)
    best_10.loc[best_10['id']==i,'id']=tt
best_10=best_10.rename(columns={'id':'title'})
#time.sleep(delay) # delay

In [11]:
best_10

,title,rating
2299,Temptation Island,10.0
2392,Streets of Rage,10.0
521,Lock Up Your Daughters!,10.0
528,The Wedding Party,10.0
148,Pigs Is Pigs,10.0
2365,Eglimata,10.0
360,"I, the Jury",10.0
1925,Little Men,10.0
704,The Great Smokey Roadblock,10.0
271,Desert Fury,10.0


In [12]:
id_worst=worst_10['id'] # id of the titles of the top 10 movies
#delay = 3  # 3-second delay between requests
for i in id_worst:
    url=url_base+i # imdb url of the movies
    tt=title(url)
    worst_10.loc[worst_10['id']==i,'id']=tt
worst_10=worst_10.rename(columns={'id':'title'})
#time.sleep(delay) # delay

In [13]:
worst_10

,title,rating
2748,Bullet Boy,1.0
2747,The Waystation: Prelude,1.0
879,Polyester,1.0
2731,Touching Evil,1.0
2730,The Panel,1.0
2729,Saturday's Warrior,1.0
2728,Public Domain,1.0
2727,Look Both Ways,1.0
2724,Harvie Krumpet,1.0
3455,No Regret,1.0
